# Modelo: XGBoost

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from xgboost import XGBRegressor

In [8]:
def calc_metricas(obs, pred):
    r2 = metrics.r2_score(obs, pred).round(3)
    mape = (100 * metrics.mean_absolute_percentage_error(obs, pred)).round(3)
    mae = metrics.mean_absolute_error(obs, pred).round(2)
    rmse = (metrics.mean_squared_error(obs, pred)**0.5).round(2)
    msle = metrics.mean_squared_log_error(obs, pred).round(3)
    
    return [r2, mape, mae, rmse, msle]

def print_avaliacao(obs, pred):
    print('R² = %.3f' % metrics.r2_score(obs, pred))
    print('MAPE = %.3f %%' % (100 * metrics.mean_absolute_percentage_error(obs, pred)))
    print('MAE = U$S %.2f' % (metrics.mean_absolute_error(obs, pred)))
    print('RMSE = U$S %.2f' % metrics.mean_squared_error(obs, pred)**0.5)
    print('MSLE = %.3f' % metrics.mean_squared_log_error(obs, pred))


In [2]:
dados_treino = pd.read_csv('../data/processed/train_data.csv')
dados_teste = pd.read_csv('../data/processed/test_data.csv')

In [3]:
le_cat1 = LabelEncoder()
le_cat2 = LabelEncoder()
le_cat3 = LabelEncoder()
le_brand = LabelEncoder()

le_cat1 = le_cat1.fit(dados_treino.category_1)
le_cat2 = le_cat2.fit(dados_treino.category_2)
le_cat3 = le_cat3.fit(dados_treino.category_3)
le_brand = le_brand.fit(dados_treino.brand_name)

dados_treino['category_1'] = le_cat1.transform(dados_treino.category_1)
dados_treino['category_2'] = le_cat2.transform(dados_treino.category_2)
dados_treino['category_3'] = le_cat3.transform(dados_treino.category_3)
dados_treino['brand_name'] = le_brand.transform(dados_treino.brand_name)

In [4]:
dic_cat1 = dict(zip(le_cat1.classes_, le_cat1.transform(le_cat1.classes_)))
dic_cat2 = dict(zip(le_cat2.classes_, le_cat2.transform(le_cat2.classes_)))
dic_cat3 = dict(zip(le_cat3.classes_, le_cat3.transform(le_cat3.classes_)))
dic_brand = dict(zip(le_brand.classes_, le_brand.transform(le_brand.classes_)))


In [5]:
dados_teste = pd.read_csv('../data/processed/test_data.csv')
dados_teste['category_1'] = dados_teste['category_1'].map(dic_cat1).fillna(-1)
dados_teste['category_2'] = dados_teste['category_2'].map(dic_cat2).fillna(-1)
dados_teste['category_3'] = dados_teste['category_3'].map(dic_cat3).fillna(-1)
dados_teste['brand_name'] = dados_teste['brand_name'].map(dic_brand).fillna(-1)
dados_teste

,name,category_1,category_2,category_3,item_condition_id,brand_name,price,shipping,item_description,date,stock
0,women s motorcycle tshirt,10,104,751.0,3,1709.0,6.0,1,xl fits like a small,9-4-2018,8
1,alex and ani necklace,10,58,530.0,2,47.0,11.0,1,path of life alex and ani necklace in gold has...,25-1-2018,8
2,steve madden red lace up heels,10,88,629.0,2,3682.0,36.0,0,never been worn isn t true to size runs small,30-11-2018,9
3,men s black tuxedo,5,94,811.0,3,2795.0,54.0,0,worn a few times single pleated pants pants ar...,30-7-2018,7
4,red open back mermaid style dress xs,10,35,334.0,1,2795.0,316.0,0,this full length wine red mermaid style backle...,29-7-2018,34
...,...,...,...,...,...,...,...,...,...,...,...
444494,pink vs sleep shorts,10,5,687.0,3,2928.0,12.0,0,size small great condition barely ever worn to...,16-12-2018,7
444495,coach hand bag,10,112,392.0,3,926.0,86.0,0,this large black hobo coach is in a very good ...,15-11-2018,3
444496,waffle knit sweater forever,10,97,226.0,3,1384.0,14.0,0,great condition tags forever american apparel ...,2-3-2018,4
444497,dr dre beats solo,1,100,381.0,3,457.0,69.0,0,blue fits any head amazing sound no longer in use,19-3-2018,6


## Modelo geral usando apenas coluna name

In [6]:
model = XGBRegressor(random_state=101, n_jobs=-1)
# model = XGBRegressor(n_estimators=1000, 
#                      max_depth=7,
#                      grow_policy='lossguide', 
#                      eta=0.1, 
#                      subsample=0.7, 
#                      colsample_bytree=0.8, 
#                      random_state=101, 
#                      n_jobs=-1)
X_treino = dados_treino[['category_1', 'category_2', 'category_3', 'brand_name', 'item_condition_id', 'shipping']]
y_treino = dados_treino['price']
model.fit(X_treino, y_treino)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100,
             n_jobs=-1, num_parallel_tree=1, predictor='auto', random_state=101,
             reg_alpha=0, reg_lambda=1, ...)

Tempo de treino: 57.2s

In [7]:
pred = model.predict(dados_teste[['category_1', 'category_2', 'category_3', 'brand_name', 'item_condition_id', 'shipping']])
for i in range(len(pred)):
    if pred[i] < 3:
        pred[i] = 3

Tempo de previsão: 1.5s

In [9]:
print_avaliacao(dados_teste['price'], pred)

R² = 0.340
MAPE = 70.232 %
MAE = U$S 13.59
RMSE = U$S 31.18
MSLE = 0.374


## Modelo por categorias: name como atributo

In [10]:
modelos = {}

for category in dados_treino.category_1.unique():
    model = XGBRegressor(n_jobs=-1, random_state=101)
    modelos[category] = model
    X_treino = dados_treino.query(f'category_1 == {category}')[['category_1', 'category_2', 'category_3', 'brand_name', 'item_condition_id', 'shipping']]
    y_treino = dados_treino.query(f'category_1 == {category}')['price']
    modelos[category].fit(X_treino, y_treino)

Tempo treinamento: 38.7s

In [34]:
dados_teste = dados_teste.assign(pred=0)
for category in dados_teste.category_1.unique():
    index = dados_teste.query(f'category_1 == {category}').index
    dados_teste.loc[index, 'pred'] = modelos[category].predict(dados_teste.loc[index][['category_1', 'category_2',
                                                                                       'category_3', 'brand_name',
                                                                                       'item_condition_id', 'shipping']])

dados_teste.loc[dados_teste.pred < 3, 'pred'] = 3

Tempo previsão: 0.7s

Resultado geral:

In [36]:
print_avaliacao(dados_teste['price'], pred)

R² = 0.340
MAPE = 70.232 %
MAE = U$S 13.59
RMSE = U$S 31.18
MSLE = 0.374


Resultado por categoria

In [35]:
avaliacoes = {}

for categoria in dados_teste.category_1.unique():
    dados_cat = dados_teste.query(f'category_1 == {categoria}')
    avaliacoes[categoria] = calc_metricas(dados_cat['price'], dados_cat['pred'])

avaliacoes = pd.DataFrame(avaliacoes, index=['R²', 'MAPE', 'MAE', 'RMSE', 'MSLE'],
                          )
avaliacoes.columns = ['Women', 'Men', 'Vintage & Collectibles', 'Electronics',
                      'Beauty', 'Kids', 'Other', 'Home', 'Sports & Outdoors',
                      'Handmade', 'No category']
avaliacoes

,Women,Men,Vintage & Collectibles,Electronics,Beauty,Kids,Other,Home,Sports & Outdoors,Handmade,No category
R²,0.359,0.477,0.239,0.410,0.081,0.251,0.111,0.184,0.221,0.189,-0.020
MAPE,59.772,58.537,96.078,91.415,64.904,64.290,81.098,65.163,74.737,72.265,91.578
MAE,13.110,15.470,17.600,21.350,9.790,10.220,12.120,12.150,13.690,10.150,15.730
RMSE,31.330,28.970,44.210,50.550,23.220,19.250,25.870,21.040,26.210,22.580,30.580
MSLE,0.309,0.302,0.554,0.509,0.341,0.332,0.446,0.351,0.416,0.379,0.557
